In [ ]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
sweep_config = {
    'method' : 'grid',
    'metric': {
        'name' : 'val_accuracy',
        'goal' : 'maximize'
    },
    'parameters' : {
        'batch_size' : { 'values' : [8, 16, 32, 64, 128]},
        'learning_rate' : { 'values' : [0.001, 0.0001, 0.00001]},
        'hidden_nodes': {'values' : [32, 64, 128, 256]},
        'img_size' : {'values' : [16, 64, 224]},
        'epochs' : {'values': [5, 10]}
    }
}
sweep_id = wandb.sweep(sweep_config, project="5-Flower-Dataset")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 4c9k46ix
Sweep URL: https://wandb.ai/fangselection123-happiest-minds-technologies/5-Flower-Dataset/sweeps/4c9k46ix


#This below code gives the structure of the code that needs to be train by using weight and bias. Code contains the simple neural network with activation function

In [ ]:
def train():
  with wandb.init() as run:
    config= wandb.config
    import tensorflow as tf
    IMG_HEIGHT= config.img_size
    IMG_WIDTH= config.img_size
    IMG_CHANNELS=3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
    def read_and_decode(filename, resize_dims):
      # 1. Read the raw file
      img_bytes= tf.io.read_file(filename)
      # 2. Decode image data
      img= tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
      # 3. Convert image to float values in [0, 1]
      img= tf.image.convert_image_dtype(img, tf.float32)
      # 4. Resize the image to the match the desire dimention
      img= tf.image.resize(img, resize_dims)
      return img

    def parse_csvline(csv_line):
      # print("csv line:", csv_line)
      # record_defaults specify the data types for each columns
      record_default = ["", ""]
      filename, label_string =tf.io.decode_csv(csv_line, record_defaults=record_default)


      #load the image
      img= read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
      # print("Label String:",label_string)
      label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
      return img, label

    #Define dataset
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        #.map(parse_csvline) # it will process one by one line to the map function which is slow
        #.map(parse_csvline, num_parallel_calls=4) # it will process one by four line to the map function which is faster

        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE) # It will adjust the number of line to the function depends upon the cpu
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    test_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE) # When the model is training the current batch then it will prepare the next batch in the background.
    )
    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)),
    keras.layers.Dense(config.hidden_nodes, activation='relu'),
    keras.layers.Dense(len(CLASS_NAMES), activation='softmax')
    ])

    model.compile(optimizer='adam', loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=config.epochs,
    callbacks=[WandbMetricsLogger()]
    )


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: ugni17k3 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 32
wandb: 	img_size: 64
wandb: 	learning_rate: 0.001


Epoch 1/5
    413/Unknown 160s 384ms/step - accuracy: 0.2203 - loss: 1.8444

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 178s 426ms/step - accuracy: 0.2203 - loss: 1.8440 - val_accuracy: 0.2351 - val_loss: 1.6076
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 341ms/step - accuracy: 0.2397 - loss: 1.6032 - val_accuracy: 0.2351 - val_loss: 1.6080
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 339ms/step - accuracy: 0.2397 - loss: 1.6018 - val_accuracy: 0.2351 - val_loss: 1.6088
Epoch 4/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 139s 333ms/step - accuracy: 0.2397 - loss: 1.6015 - val_accuracy: 0.2351 - val_loss: 1.6095
Epoch 5/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 331ms/step - accuracy: 0.2397 - loss: 1.6015 - val_accuracy: 0.2351 - val_loss: 1.6099


epoch/accuracy,▁████
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▁▁▁▁
epoch/val_accuracy,▁▁▁▁▁
epoch/val_loss,▁▂▅▇█
epoch/accuracy,0.24576
epoch/epoch,4
epoch/learning_rate,0.001
epoch/loss,1.59942
epoch/val_accuracy,0.23514


...

wandb: Agent Starting Run: jgkw1wwk with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001


Epoch 1/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 343ms/step - accuracy: 0.3105 - loss: 14.5916 - val_accuracy: 0.3189 - val_loss: 3.0101
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 144s 349ms/step - accuracy: 0.2597 - loss: 2.1471 - val_accuracy: 0.2351 - val_loss: 1.6075
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 200s 485ms/step - accuracy: 0.2397 - loss: 1.6045 - val_accuracy: 0.2351 - val_loss: 1.6074
Epoch 4/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 202s 486ms/step - accuracy: 0.2397 - loss: 1.6023 - val_accuracy: 0.2351 - val_loss: 1.6081
Epoch 5/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 484ms/step - accuracy: 0.2397 - loss: 1.6016 - val_accuracy: 0.2351 - val_loss: 1.6089


epoch/accuracy,█▁▁▁▁
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▁▁▁▁
epoch/val_accuracy,█▁▁▁▁
epoch/val_loss,█▁▁▁▁
epoch/accuracy,0.24576
epoch/epoch,4
epoch/learning_rate,0.001
epoch/loss,1.59977
epoch/val_accuracy,0.23514


wandb: Agent Starting Run: 1ex0dikw with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.0001


Epoch 1/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 144s 347ms/step - accuracy: 0.2152 - loss: 8.5075 - val_accuracy: 0.2351 - val_loss: 1.6079
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 342ms/step - accuracy: 0.2397 - loss: 1.6035 - val_accuracy: 0.2351 - val_loss: 1.6080
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 144s 347ms/step - accuracy: 0.2397 - loss: 1.6019 - val_accuracy: 0.2351 - val_loss: 1.6087
Epoch 4/5
291/413 ━━━━━━━━━━━━━━━━━━━━ 40s 336ms/step - accuracy: 0.2370 - loss: 1.6025